In [3]:
import pandas as pd
import os
import re
from tqdm import tqdm 
import matplotlib.pyplot as plt
import numpy as np
# import warnings
import pickle
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
# import math
import torch.nn.functional as F
# from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import Dataset, DataLoader
# import time
import wandb
import torch.optim as optim
import random
from util import accuracy
from sklearn.metrics import (confusion_matrix, accuracy_score, classification_report,
                             auc, roc_auc_score,average_precision_score)
from training_util_mae import (MyDataset, split, Transformer_park, Transformer_pretrain,
                        base_model, base_decoder, hierachy_model, train, validate, weighted_sampling,
                              eval_model)
import easydict
from sklearn.preprocessing import label_binarize


### define model training

In [2]:
def model_train(args):
    b_model = base_model(n_each_base = args.n_each_base,
                         mask_on=args.mask_on).cuda(args.cuda_i)
    vital_model = Transformer_park(n_each_lab = args.n_each_vital,
                                       n_embed = args.n_embed, 
                                       nhead = args.nhead, 
                                       nhid = args.nhid, 
                                       nlayers = args.nlayers, 
                                       dropout = args.dropout, 
                                       mask_on = args.mask_on).cuda(args.cuda_i)
    lab_model = Transformer_park(n_each_lab = args.n_each_lab,
                                     n_embed = args.n_embed, 
                                     nhead = args.nhead, 
                                     nhid = args.nhid, 
                                     nlayers = args.nlayers, 
                                     dropout = args.dropout, 
                                     mask_on = args.mask_on ).cuda(args.cuda_i)
    decoder_vital = Transformer_pretrain(n_each_lab = args.n_each_vital,
                                        n_embed = args.n_embed).cuda(args.cuda_i)
    decoder_lab = Transformer_pretrain(n_each_lab = args.n_each_lab,
                                       n_embed = args.n_embed).cuda(args.cuda_i)
    decoder_base = base_decoder(n_each_base = args.n_each_base).cuda(args.cuda_i)
    total_model = hierachy_model(vital_shape = args.vital_shape, 
                                 lab_shape = args.lab_shape, 
                                 batch_size =args.batch_size, 
                                 output_len = args.output_len).cuda(args.cuda_i)
    optimizer = optim.Adam([{"params":vital_model.parameters()},
                            {"params":lab_model.parameters()},
                            {"params":b_model.parameters()},
                            {"params":decoder_base.parameters()},
                            {"params":decoder_vital.parameters()},
                            {"params":decoder_lab.parameters()},
                           {"params":total_model.parameters()}], 
                            lr = args.lr_, 
                            weight_decay = args.weight_decay_)
    wandb.watch([b_model, vital_model, lab_model,total_model])   
    best_auroc = -np.inf
    best_loss = np.inf

    for epoch in range(1, args.epochs):
        train_loss = train(train_loader, b_model, vital_model, lab_model, decoder_vital,
          decoder_lab, decoder_base, total_model, optimizer, 
                           weight = args.weight, 
                              masking_value = args.masking_value, 
                               mae = args.mae, 
                               masking_ratio = args.masking_ratio,
                               i = args.cuda_i)
        
        val_total_loss, val_o_loss, roc_weight_val = validate(valid_loader, b_model, vital_model,
                                                              lab_model, decoder_vital,
          decoder_lab, decoder_base, total_model, optimizer, weight = args.weight,
                                                                  i = args.cuda_i)
        
        
        if args.metric == "auroc" and roc_weight_val >= best_auroc:
            filename = save_function(args.save_path, args.kfold_num,
                                     b_model, vital_model, lab_model, decoder_vital,
          decoder_lab, decoder_base, total_model, optimizer)
            epoch_no_improve = 0
            best_auroc = roc_weight_val
        elif args.metric == "total_loss" and val_total_loss<= best_loss :
            filename = save_function(args.save_path, args.kfold_num,
                                    b_model, vital_model, lab_model, decoder_vital,
          decoder_lab, decoder_base, total_model, optimizer)
            epoch_no_improve = 0
            best_loss = val_total_loss
        elif args.metric == "target_loss" and val_o_loss<= best_loss :
            filename = save_function(args.save_path, args.kfold_num,
                                    b_model, vital_model, lab_model, decoder_vital,
          decoder_lab, decoder_base, total_model, optimizer)
            epoch_no_improve = 0
            best_loss = val_o_loss
        else:
            epoch_no_improve +=1
            
        ###early stopping
        if args.early_stop:
            if epoch> args.min_epoch and epoch_no_improve > args.n_epochs_stop:
                break
            else:
                continue

    return filename

def save_function(save_path, number, b_model, vital_model, lab_model, decoder_vital,
          decoder_lab, decoder_base, total_model, optimizer):
    fileuser = [f for f in os.listdir(save_path) 
                if f.startswith(f"model{number}_pad_hos{padded_hosday}_emb_{args.n_embed}_\
    hid_{args.nhid}_layer_{args.nlayers}_lr_{args.lr_}_mr{args.masking_ratio}")]
    if fileuser:
        if os.path.isfile(save_path+"/"+fileuser[0]):
            os.remove(save_path+"/"+fileuser[0])

    filename = save_path+f"/model{number}_pad_hos{padded_hosday}_emb_{args.n_embed}_\
    hid_{args.nhid}_layer_{args.nlayers}_lr_{args.lr_}_mr{args.masking_ratio}.pt"
    torch.save({"b_model_dict":b_model.state_dict(),
               "vital_model_dict":vital_model.state_dict(),
                "lab_model_dict":lab_model.state_dict(),
                "decoder_vital_dict":decoder_vital.state_dict(),
               "decoder_lab_dict":decoder_lab.state_dict(),
                "decoder_base_dict":decoder_base.state_dict(),
               "total_model_dict":total_model.state_dict(),
               "optimizer_dict":optimizer.state_dict()}, filename)
    return filename

In [3]:
def loading_model(args, loaded_model):
    b_model = base_model(n_each_base = args.n_each_base,
                         mask_on=args.mask_on)
    vital_model = Transformer_park(n_each_lab = args.n_each_vital,
                                       n_embed = args.n_embed, 
                                       nhead = args.nhead, 
                                       nhid = args.nhid, 
                                       nlayers = args.nlayers, 
                                       dropout = args.dropout, 
                                       mask_on = args.mask_on)
    lab_model = Transformer_park(n_each_lab = args.n_each_lab,
                                     n_embed = args.n_embed, 
                                     nhead = args.nhead, 
                                     nhid = args.nhid, 
                                     nlayers = args.nlayers, 
                                     dropout = args.dropout, 
                                     mask_on = args.mask_on )
    decoder_vital = Transformer_pretrain(n_each_lab = args.n_each_vital,
                                        n_embed = args.n_embed)
    decoder_lab = Transformer_pretrain(n_each_lab = args.n_each_lab,
                                       n_embed = args.n_embed)
    decoder_base = base_decoder(n_each_base = args.n_each_base)
    total_model = hierachy_model(vital_shape = args.vital_shape, 
                                 lab_shape = args.lab_shape, 
                                 batch_size =args.test_batchsize, 
                                 output_len = args.output_len)

    vital_model.load_state_dict(loaded_model['vital_model_dict'])
    lab_model.load_state_dict(loaded_model['lab_model_dict'])
    b_model.load_state_dict(loaded_model['b_model_dict'])
    total_model.load_state_dict(loaded_model['total_model_dict'])
    decoder_vital.load_state_dict(loaded_model['decoder_vital_dict'])
    decoder_lab.load_state_dict(loaded_model['decoder_lab_dict'])
    decoder_base.load_state_dict(loaded_model['decoder_base_dict'])
    
    vital_model.eval()
    lab_model.eval()
    b_model.eval()
    total_model.eval()
    decoder_vital.eval()
    decoder_lab.eval()
    decoder_base.eval()

    return vital_model, lab_model, b_model, total_model, decoder_vital, decoder_lab, decoder_base

In [4]:
def predicted_outcome(test_loader, vital_model, lab_model, b_model, total_model, 
                decoder_vital, decoder_lab, decoder_base):
    for batch, batch_data in enumerate(test_loader):
        vital_set = batch_data[0]
        lab_set = batch_data[1]
        baseline_set = batch_data[2]
        each_att_vital_set = batch_data[3]
        each_att_lab_set = batch_data[4]
        each_att_base_set = batch_data[5]
        att_vital_set = batch_data[6]
        att_lab_set = batch_data[7]
        outcome = batch_data[8]
        outcome = outcome.type(torch.LongTensor)

        vital_tensor = vital_model(vital_set.transpose(0,1), att_vital_set,
                                   each_mask = each_att_vital_set.transpose(0,1))
        lab_tensor = lab_model(lab_set.transpose(0,1), att_lab_set,
                               each_mask = each_att_lab_set.transpose(0,1))
        base_tensor = b_model(baseline_set, each_att_base_set)

        vital_out = decoder_vital(vital_tensor)  #MLM용도
        lab_out = decoder_lab(lab_tensor)    #MLM 용도
        base_out = decoder_base(base_tensor) #MLM 용도

        #compute loss
        result = total_model(vital_tensor, lab_tensor, base_tensor)
        
        if batch==0:
            result_t = result.cpu().detach()
            outcome_t =outcome.cpu().detach()
        else:
            result_t = torch.cat([result.cpu(), result_t],axis=0).detach()
            outcome_t = torch.cat([outcome.cpu(), outcome_t],axis=0).detach()
            
    return result_t, outcome_t

In [5]:
def auroc_auprc(predicted, outcome):
    val_acc1, val_acc2 = accuracy(predicted, outcome, topk=(1,2))
    soft = nn.Softmax(dim=1)
    s_predicted = soft(predicted).detach().numpy()
    outcome_onehot = label_binarize(outcome, classes=[0,1,2])

    avscore = average_precision_score(outcome_onehot, s_predicted, average='weighted')
    val_acc1 = val_acc1.item()
    roc_weight_val = roc_auc_score(outcome_onehot, s_predicted, 
                                   multi_class= 'ovr', average = 'weighted')

    return val_acc1, roc_weight_val, avscore

In [6]:
args = easydict.EasyDict({"masking_value": 0, #masking value selection 
                          "batch_size": 1000, #number of each batch 
                          "file_name": "../whole_3data0630_o2_idscale.txt",
                          "min_epoch": 200, #minimum epoch for training
                          "n_epochs_stop": 10, #stop epoch after nth training without performance gain
                          "epochs": 1000, #maximum epoch 
                          "early_stop": True, #setting of early stopping
                          "nhead": 4,  #transformer number of head
                          "n_embed": 160, #transformer number of embedding layer
                          "nhid": 320, #transformer number of hidden layer
                          "nlayers": 4, #transformenr encoder layer number
                          "dropout": 0.2, #drop out 
                          "lr_": 5e-5, #learning rate
                          "weight_decay_": 1e-4, #weight decay
                          "output_len": 3, #number of output category number
                          "prior_d":0, #prediction day 0 or day 1 or day 2
                          "weight": 1.0, #weight of decoder loss
                          "mae": False, #use of masked autoencoder 
                          "mask_on":True, #use of masking layer
                          "masking_ratio": 0., #MAE ratio when it used
                          "metric": 'total_loss', #early stopping reference 
                          "cuda_i" : 1, #cuda device number
                            "save_path" : "./save_confidence_interval" #save folder name 
                          })

In [8]:
# data shape과 관련된 hyperparameters

padded_hosday_list = [2,4,6,8,10,12,14,16,18,20]
if not os.path.isdir(args.save_path):
    os.makedirs(args.save_path)

In [ ]:
#MLM 시작전 모델 loss 
performance = dict()
accu_list = list()
auroc_list = list()
auprc_list = list()
padd_list = list()
for padded_hosday in padded_hosday_list:

    for number in range(5):
        args.kfold_num = number
        train_set, valid_set, test_set = split(args.file_name, random_s=number) 

        train_dataset = MyDataset(args.masking_value, train_set, 
                                  padded_hosday, data_pr = 1,prior_d=args.prior_d)
        valid_dataset = MyDataset(args.masking_value, valid_set, 
                                  padded_hosday, data_pr = 1, prior_d=args.prior_d)
        test_dataset = MyDataset(args.masking_value, test_set, 
                                 padded_hosday, data_pr = 1, prior_d=args.prior_d)

        train_loader, valid_loader = weighted_sampling(train_dataset, valid_dataset, 
                                                       batch_size = args.batch_size)
        
        args.n_each_vital = train_set[0][2].shape[-1]
        args.n_each_lab = train_set[0][1].shape[-1]
        args.n_each_base = train_set[0][0].shape[-1]

        args.vital_shape = padded_hosday*3*args.n_embed
        args.lab_shape = padded_hosday*args.n_embed
        args.test_batchsize = len(test_dataset)
        wandb.init(project="brmh0728_3_today_o2", entity='podkd', 
                   tags=[f'padding day:{padded_hosday}'])
        wandb.run.name = f"model_e:pad_day_{padded_hosday}_number{number}"

        filename = model_train(args)
        loaded_model_0 = torch.load(filename)
        
        test_loader = DataLoader(test_dataset, batch_size=len(test_dataset),
                                     shuffle=False, drop_last=True)
        load_0 = loading_model(args, loaded_model_0)
        predicted, outcome = predicted_outcome(test_loader, load_0[0], load_0[1],
                                       load_0[2],load_0[3], load_0[4], load_0[5], load_0[6])
        
        accura1, auroc_test, auprc_test = auroc_auprc(predicted, outcome)
        accu_list.append(accura1)
        auroc_list.append(auroc_test)
        auprc_list.append(auprc_test)
        padd_list.append(padded_hosday)

performance["accuracy"] = accu_list
performance["auroc"] = auroc_list
performance["auprc"] = auprc_list
performance["padding_day"] = padd_list
    
performance = pd.DataFrame(performance)

load completed


100%|██████████| 2401/2401 [00:04<00:00, 507.27it/s]


time: 1.2385096549987793 stack complete
time: 0.0029115676879882812 dataset completely loaded
whole data is completely loaded
load completed


100%|██████████| 801/801 [00:01<00:00, 641.99it/s]


time: 0.265444278717041 stack complete
time: 0.0007102489471435547 dataset completely loaded
whole data is completely loaded
load completed


...

100%|██████████| 801/801 [00:01<00:00, 477.34it/s]


time: 0.49842238426208496 stack complete
time: 0.002262115478515625 dataset completely loaded
whole data is completely loaded
load completed


100%|██████████| 801/801 [00:01<00:00, 482.12it/s]


time: 0.47652673721313477 stack complete
time: 0.0021517276763916016 dataset completely loaded
whole data is completely loaded


auroc_micro,▁▅▆▆▇▇▇▇▇▇▇██████████▇█████▇████████▇███
auroc_weight,▁▅▆▆▇▇▇▇▇▇▇███▇██████▇██████████████████
top1 accuracy,▁▅▅▅▆▆▇▇▇▇▇▇▇▇▇██████▇██████████████████
train outcome loss,█▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train total loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val outcome loss,█▄▃▃▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂▃▂▁▃▂▂▃▃▃▃▃▃▄▃▄▃
val total loss,█▄▃▃▂▂▂▂▂▂▁▁▁▁▂▁▂▂▂▁▂▂▁▂▂▂▁▂▂▂▃▂▂▃▂▂▃▂▃▂
auroc_micro,0.91729
auroc_weight,0.92409
top1 accuracy,79.5625
train outcome loss,0.3413


In [ ]:
performance.to_csv("./performance.csv")

In [4]:
pd.read_csv("./performance.csv")

,Unnamed: 0,accuracy,auroc,auprc,padding_day
0,0,75.687080,0.899842,0.875173,2
1,1,73.708069,0.894894,0.868742,2
2,2,79.292404,0.915219,0.899777,2
3,3,74.401070,0.887667,0.866931,2
4,4,75.101028,0.894946,0.875554,2
5,5,79.341988,0.922428,0.899201,4
6,6,76.838776,0.906892,0.885279,4
7,7,81.234375,0.926430,0.905338,4
8,8,79.321434,0.919866,0.896227,4
9,9,77.535744,0.909147,0.879303,4
